# New York City Department of Health Data

Augmenting the addresses with Open Maps data using [``geopy`` on PyPi](https://pypi.python.org/pypi/geopy).

In [1]:
import pandas, bokeh, bokeh.plotting, geopy, geopy.geocoders

In [93]:
df = pandas.read_csv('DOHNYC.csv', low_memory=False)

In [103]:
df.head(2)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462,7188924968,Bakery,2015-02-09,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,6,A,2015-02-09,2015-08-14,Cycle Inspection / Initial Inspection
1,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462,7188924968,Bakery,2014-03-03,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,2,A,2014-03-03,2015-08-14,Cycle Inspection / Initial Inspection


In [94]:
# Compose address string for OpenMaps
addresses = False
for col in ['BUILDING','STREET','BORO','NY','ZIPCODE']:
    if not isinstance( addresses, pandas.Series):
        addresses = df[col].apply( lambda v: str(v)) + ' ' 
    else:
        addresses += (df[col].apply( lambda v: str(v)) if col in df.columns else col )+ ' ' 
addresses = addresses.unique() 

In [102]:
# Request the augmented data
geocoder = geopy.geocoders.Nominatim()
addy_mapper = {}
for address in addresses[:10]:
    location = geocoder.geocode( address, addressdetails=True, language=True, )
    if location and hasattr(location,'raw'):
        addy_mapper[address] =  location.raw
        for k, v in location.raw['address'].items():
            addy_mapper[address][k] = v
        del addy_mapper[address]['address']

In [101]:
pandas.DataFrame(addy_mapper).transpose().to_csv('address.csv')